In [ ]:
!wget https://raw.githubusercontent.com/adityatripathiiit/temporary/main/cc_data.csv

--2021-02-09 16:51:41--  https://raw.githubusercontent.com/adityatripathiiit/temporary/main/cc_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 902879 (882K) [text/plain]
Saving to: ‘cc_data.csv’

cc_data.csv         100%[===================>] 881.72K  --.-KB/s    in 0.07s   

2021-02-09 16:51:41 (12.6 MB/s) - ‘cc_data.csv’ saved [902879/902879]



**Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd
import random
from tabulate import tabulate 
import sys

**Function to Normalize the data** 

In [ ]:
def normalize(cc_data): 
    min = cc_data.min()
    max = cc_data.max()
    cc_data = (cc_data- min)/(max-min)
    return (cc_data)

**L2 Metric Function Between 2 points**

In [ ]:
def L2(p1,p2):
    return (np.linalg.norm(p1-p2))

**Function to precompute all the pair wise L2 distances between all the points in dataset**

In [ ]:
def dist_precompute(cc_data):
    n = len(cc_data)
    dist = [[0 for i in range(n)] for j in range(n)]
    for xi in range(n):
        for xj in range(xi,n):
            dist[xi][xj] = L2(cc_data[xi], cc_data[xj])
            dist[xj][xi] = dist[xi][xj]
    return (dist)

**Function for Greedy K-center Algorithm (2-approx)**

In [ ]:
def max_dist(curr_cluster, cc_data):
    i_max = -1;
    n = len(cc_data)
    MAX = -1
    for xi in range(n):
        ci = curr_cluster[xi]
        dist = L2(cc_data[ci], cc_data[xi])
        if dist > MAX :
            MAX = dist 
            i_max = xi 
    return (i_max, MAX)
def k_center_greedy(cc_data, k):
    n = len(cc_data)
    random_center = random.randint(0, n-1)
    # centers = [random_center]
    k -= 1
    curr_cluster = [random_center for i in range(n)]
    while k:
        curr_center, cost = max_dist(curr_cluster, cc_data)
        # centers.append(curr_center)
        for i in range(n):
            if L2(cc_data[curr_center], cc_data[i]) < L2(cc_data[curr_cluster[i]], cc_data[i]):
                curr_cluster[i] = curr_center
        k -= 1
    return (cost)

**Function for Optimal K-center for small values of K (2 or 4)**

In [ ]:
def k2_center_opt(dist):
    n = len(dist)
    final_centers = []
    cost = sys.maxsize 
    for c1 in range(n):
        for c2 in range(c1+1,n):
            curr_cost = -sys.maxsize
            for xi in range(n):
                if (xi!= c1 and xi != c2):
                    curr_cost = max(min(dist[c1][xi], dist[c2][xi]), curr_cost)
            if curr_cost <cost:
                cost = curr_cost
                final_centers = [c1,c2]
    return (cost)
def k4_center_opt(dist):
    n = len(dist)
    final_centers = []
    cost = sys.maxsize
    for c1 in range(n):
        for c2 in range(c1+1,n):
            for c3 in range(c2+1, n):
                for c4 in range(c3+1, n):
                    curr_cost = -sys.maxsize
                    for xi in range(n):
                        if (xi!= c1 and xi != c2 and xi!= c3 and xi != c4):
                            curr_cost = max(min(dist[c1][xi], dist[c2][xi], dist[c3][xi], dist[c4][xi]), curr_cost)
                    if curr_cost <cost:
                        cost = curr_cost
                        final_centers = [c1,c2,c3,c4]
    return (cost)

**Reading CSV file Using Pandas and Normalizing the data**

In [ ]:
cc_data = normalize(pd.read_csv("./cc_data.csv").fillna(0).set_index("CUST_ID"))

**Running the Greedy K-means Algorithm On the complete dataset**

In [ ]:
k_values = [2,4,10]
res = []
cc_data_arr = cc_data.to_numpy()
for k in k_values:
    cost = k_center_greedy(cc_data_arr,k)
    res.append([k,cost])
out = pd.DataFrame(res, columns = ['K', 'Cost']).set_index('K')
print(tabulate(out, headers = 'keys', tablefmt = 'fancy_grid'))
# print(cc_data.sample(5))

╒═════╤═════════╕
│   K │    Cost │
╞═════╪═════════╡
│   2 │ 2.28861 │
├─────┼─────────┤
│   4 │ 1.92486 │
├─────┼─────────┤
│  10 │ 1.46966 │
╘═════╧═════════╛


**Reducing dataset size significantly and Precomputing Distance for optimal algorithm**
<br> Note: The sample size is just 50 rows/datapoints, it is because, the when k = 4, then the optimal algorithm will take forever to complete on a larger dataset. 

In [ ]:
cc_data_short = cc_data.sample(50)
cc_data_short = cc_data_short.to_numpy()
dist = dist_precompute(cc_data_short)

**Running the optimal algorithm for k-center(2 and 4)**

In [ ]:
cost_k2_opt = k2_center_opt(dist)   
cost_k4_opt = k4_center_opt(dist)
res_opt = [[2,cost_k2_opt], [4,cost_k4_opt]]
out_opt = pd.DataFrame(res_opt, columns = ['K', 'Cost-Optimal']).set_index('K')
print(tabulate(out_opt, headers = 'keys', tablefmt = 'fancy_grid'))

╒═════╤════════════════╕
│   K │   Cost-Optimal │
╞═════╪════════════════╡
│   2 │        1.2275  │
├─────┼────────────────┤
│   4 │        1.04296 │
╘═════╧════════════════╛


**Greedy K-means For smaller data set**

Note: It wont be fare to compare the ratio of optimal on a very small dataset and greedy on a large dataset. Therefore, I also compare ratio of both of them for smaller datasets also .

In [ ]:
res_50 = []
for k in k_values:
    cost = k_center_greedy(cc_data_short,k)
    res_50.append([k,cost])
out = pd.DataFrame(res_50, columns = ['K', 'Cost']).set_index('K')
print(tabulate(out, headers = 'keys', tablefmt = 'fancy_grid'))

╒═════╤══════════╕
│   K │     Cost │
╞═════╪══════════╡
│   2 │ 1.75967  │
├─────┼──────────┤
│   4 │ 1.32503  │
├─────┼──────────┤
│  10 │ 0.749823 │
╘═════╧══════════╛


**Approximation factor** 

In [ ]:
factor_k2 = res[0][1]/res_opt[0][1]
factor_k2_50 = res_50[0][1]/res_opt[0][1]
factor_k4 = res[1][1]/res_opt[1][1]
factor_k4_50 = res_50[1][1]/res_opt[1][1]
factor = [[2,res[0][1], res_opt[0][1],res_50[0][1], factor_k2, factor_k2_50],[4,res[1][1], res_opt[1][1],res_50[1][1], factor_k4, factor_k4_50]]
fact_out = pd.DataFrame(factor, columns = ['K','Cost-approx(Full entries)','Cost-approx(50 entries)', 'Cost-optimal(50 enties)', 'Approximation Factor(full/50 entries)', 'Approximation Factor(50/50 entries)']).set_index('K')
print(tabulate(fact_out, headers = 'keys', tablefmt = 'fancy_grid'))

╒═════╤═════════════════════════════╤═══════════════════════════╤═══════════════════════════╤═════════════════════════════════════════╤═══════════════════════════════════════╕
│   K │   Cost-approx(Full entries) │   Cost-approx(50 entries) │   Cost-optimal(50 enties) │   Approximation Factor(full/50 entries) │   Approximation Factor(50/50 entries) │
╞═════╪═════════════════════════════╪═══════════════════════════╪═══════════════════════════╪═════════════════════════════════════════╪═══════════════════════════════════════╡
│   2 │                     2.28861 │                   1.2275  │                   1.75967 │                                 1.86445 │                               1.43354 │
├─────┼─────────────────────────────┼───────────────────────────┼───────────────────────────┼─────────────────────────────────────────┼───────────────────────────────────────┤
│   4 │                     1.92486 │                   1.04296 │                   1.32503 │                           

**Conclusion** 

The approximate algorithm is a 2 approximate algorithm, therefore the cost of the approximate algorithm should be less than 2*cost of optimal algorithm. 
Note that we should always compare this result on a dataset of same size.
Therefore, in the above table, the Approximation Factor(50/50 entries) makes much more sense and gives a better idea of the approximate factor. 